In [4]:
# from flask import Flask, request, url_for, render_template, redirect, abort, flash,jsonify
import psycopg2
import pandas as pd
from config import aws
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# from imblearn.ensemble import BalancedRandomForestClassifier
import pickle

# gets the text from the webpage
home_team = "TOR"
away_team = "DAL"

# generate the table from your two teams
print(home_team, away_team)

# Import data from postgresql
conn = psycopg2.connect(
database="NBA_ML",
user="root",
password=aws,
host="root.cv2swqkze1kh.us-east-1.rds.amazonaws.com",
port='5432'
)

games_df = pd.read_sql_query('select * from game_detail_new',con=conn)

games_df

TOR DAL


,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,fgm,...,reb,ast,stl,blk,tov,pf,pts,plus_minus,video_available,win
0,22021,1610612746,LAC,LA Clippers,22101083,2022-03-22,LAC @ DEN,L,240,42,...,35,27,3,7,6,19,115,-12,1,0
1,22021,1610612743,DEN,Denver Nuggets,22101083,2022-03-22,DEN vs. LAC,W,240,45,...,40,31,4,5,11,19,127,12,1,1
2,22021,1610612749,MIL,Milwaukee Bucks,22101082,2022-03-22,MIL vs. CHI,W,240,50,...,53,26,4,6,15,12,126,28,1,1
3,22021,1610612741,CHI,Chicago Bulls,22101082,2022-03-22,CHI @ MIL,L,240,39,...,33,21,9,4,7,12,98,-28,1,0
4,22021,1610612737,ATL,Atlanta Hawks,22101081,2022-03-22,ATL @ NYK,W,240,40,...,43,26,7,5,14,18,117,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23655,22012,1610612748,MIA,Miami Heat,21200002,2012-10-30,MIA vs. BOS,W,240,43,...,36,25,8,5,8,20,120,13,1,1
23656,22012,1610612764,WAS,Washington Wizards,21200001,2012-10-30,WAS @ CLE,L,240,32,...,39,26,11,10,13,19,84,-10,1,0
23657,22012,1610612739,CLE,Cleveland Cavaliers,21200001,2012-10-30,CLE vs. WAS,W,240,36,...,54,22,7,5,21,21,94,10,1,1
23658,22012,1610612747,LAL,Los Angeles Lakers,21200003,2012-10-30,LAL vs. DAL,L,240,38,...,46,24,6,5,14,21,91,-8,1,0


In [11]:
trans_df = pd.DataFrame()

# row list for df
row = []

#Last 10 games df for both teams
last_10h = games_df[games_df["team_abbreviation"] == home_team]
last_10h.sort_values("game_date", ascending=False)
#[games_df["game_date"]< games_df["game_date"]].head(10)
last_10a = games_df[games_df["team_abbreviation"] == away_team]
#[games_df["game_date"]< games_df["game_date"]].head(10)

last_10a.sort_values("game_date", ascending=False)

last_10h = last_10h.head(10)
last_10a = last_10a.head(10)
last_10h

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,fgm,...,reb,ast,stl,blk,tov,pf,pts,plus_minus,video_available,win
14,22021,1610612761,TOR,Toronto Raptors,22101076,2022-03-21,TOR @ CHI,L,240,40,...,49,26,5,6,14,20,99,-14,1,0
34,22021,1610612761,TOR,Toronto Raptors,22101069,2022-03-20,TOR @ PHI,W,240,35,...,56,16,5,4,9,16,93,5,1,1
65,22021,1610612761,TOR,Toronto Raptors,22101051,2022-03-18,TOR vs. LAL,L,265,49,...,46,25,8,2,12,19,123,-5,1,0
91,22021,1610612761,TOR,Toronto Raptors,22101044,2022-03-16,TOR @ LAC,W,240,38,...,43,14,5,1,11,12,103,3,1,1
113,22021,1610612761,TOR,Toronto Raptors,22101028,2022-03-14,TOR @ LAL,W,240,39,...,64,24,8,9,13,20,114,11,1,1
148,22021,1610612761,TOR,Toronto Raptors,22101009,2022-03-12,TOR @ DEN,W,240,48,...,50,28,12,6,8,23,127,12,1,1
165,22021,1610612761,TOR,Toronto Raptors,22101003,2022-03-11,TOR @ PHX,W,240,42,...,48,19,11,4,17,19,117,5,1,1
190,22021,1610612761,TOR,Toronto Raptors,22100989,2022-03-09,TOR @ SAS,W,240,41,...,47,12,7,1,10,23,119,15,1,1
241,22021,1610612761,TOR,Toronto Raptors,22100962,2022-03-06,TOR @ CLE,L,240,37,...,45,17,13,3,7,20,96,-8,1,0
270,22021,1610612761,TOR,Toronto Raptors,22100458,2022-03-04,TOR vs. ORL,L,240,37,...,54,22,2,8,12,18,97,-6,1,0


In [12]:
last_10a

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,fgm,...,reb,ast,stl,blk,tov,pf,pts,plus_minus,video_available,win
9,22021,1610612742,DAL,Dallas Mavericks,22101079,2022-03-21,DAL vs. MIN,W,240,40,...,47,27,5,3,12,21,110,2,1,1
49,22021,1610612742,DAL,Dallas Mavericks,22101059,2022-03-19,DAL @ CHA,L,240,39,...,44,18,4,4,15,26,108,-21,1,0
74,22021,1610612742,DAL,Dallas Mavericks,22101046,2022-03-18,DAL @ PHI,L,240,37,...,40,21,4,3,12,17,101,-10,1,0
81,22021,1610612742,DAL,Dallas Mavericks,22101036,2022-03-16,DAL @ BKN,W,240,41,...,37,21,5,2,11,17,113,2,1,1
134,22021,1610612742,DAL,Dallas Mavericks,22101014,2022-03-13,DAL @ BOS,W,240,33,...,42,20,7,5,18,16,95,3,1,1
173,22021,1610612742,DAL,Dallas Mavericks,22100999,2022-03-11,DAL @ HOU,W,240,40,...,47,20,6,8,14,21,113,13,1,1
189,22021,1610612742,DAL,Dallas Mavericks,22100988,2022-03-09,DAL vs. NYK,L,240,27,...,37,11,3,4,8,24,77,-30,1,0
225,22021,1610612742,DAL,Dallas Mavericks,22100972,2022-03-07,DAL vs. UTA,W,240,39,...,43,19,9,5,9,21,111,8,1,1
259,22021,1610612742,DAL,Dallas Mavericks,22100954,2022-03-05,DAL vs. SAC,W,240,40,...,39,22,7,2,10,22,114,1,1,1
284,22021,1610612742,DAL,Dallas Mavericks,22100943,2022-03-03,DAL vs. GSW,W,240,45,...,32,29,7,1,10,16,122,9,1,1


In [13]:
win_rateH = 0
win_rateA = 0

needed_features = ["fga", "fg_pct", "fg3_pct", "dreb", "reb", "ast"]

for feature in needed_features:
    row.append(round((sum(last_10h[feature])/10),2))
    row.append(round((sum(last_10a[feature])/10),2))

row.append(round((len(last_10h[last_10h["wl"]=="W"])/10),2))
row.append(round((len(last_10a[last_10a["wl"]=="W"])/10),2))

trans_df = trans_df.append(pd.DataFrame([row]), ignore_index=True)

trans_df=trans_df.set_axis(["fga_home","fga_away", "fg_pct_home", "fg_pct_away", "fg3_pct_home", "fg3_pct_away", 
                        "dreb_home", "dreb_away", "reb_home", "reb_away", "ast_home", "ast_away", "win_rate_home", "win_rate_away"], axis=1)

trans_df

,fga_home,fga_away,fg_pct_home,fg_pct_away,fg3_pct_home,fg3_pct_away,dreb_home,dreb_away,reb_home,reb_away,ast_home,ast_away,win_rate_home,win_rate_away
0,92.8,84.0,0.44,0.45,0.32,0.35,34.6,31.4,50.2,40.8,20.3,20.8,0.6,0.7


In [14]:
 #Creating input data
X_df = trans_df

# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_df)

#Scale the input
X_scaled = X_scaler.transform(X_df)

X_df.columns= X_df.columns.str.strip().str.upper()

# read in your model
filename = 'finalized_RF_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# pass in the data into the model
prob = loaded_model.predict_proba(X_df)
prediction = loaded_model.predict(X_df)

# result from model is 1
print(f'prob:{prob}, prediction:{prediction}')

if prediction[0] == 1:
    result = "Home team wins"
else:
    result = "Away team wins"

print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'finalized_RF_model.sav'

In [15]:
0.61*100

61.0